----

### Import CIMantic Graphs Library and CIM profile

The CIMantic Graphs library can be installed using `pip install cim-graph` and imported into any script:

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
# import cimgraph.utils as utils
import cimbuilder.utils as utils

CIMantic Graphs can support multiple CIM profiles, so it is necessary to specify which one to use. This demonstration will use the extensions from CIMHub feature/SETO, which is derived from CIM17V38.

In [2]:
# Specify CIM Profile
import importlib
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

To create a new model "from scratch", no filename or databased is specified. This connection object contains all information regarding the CIM version, serilaization format, etc.

In [3]:
# Create an empty connection for new file
params = ConnectionParameters(filename=None,
                              cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

For this demonstration, the IEEE 13 and IEEE 13 assets feeders will be used to populate each substation. It is possible to load these files from a database, or open them directly from the XML using CIMantic Graphs. Since end-users of the Grid Atlas may not have a database, the feeders will be loaded from the standalone files:

In [4]:
# # Import 13 bus model from XML file
# SFO1_feeder = cim.Feeder(mRID = 'accdf54c-c90c-4c41-9d7a-5e5d3d17a4e5')
# params = ConnectionParameters(filename='../test_models/accdf54c-c90c-4c41-9d7a-5e5d3d17a4e5.xml',
#                                cim_profile=cim_profile, iec61970_301=8)
# SFO1_connection = RDFlibConnection(params)
# SFO1_network = FeederModel(connection=SFO1_connection, container=SFO1_feeder,
#                               distributed=False)


In [5]:

# SFO2_feeder = cim.Feeder(mRID = 'bcbcaeb9-5209-49c0-b85b-417d76618ac3')
# params = ConnectionParameters(filename='../test_models/bcbcaeb9-5209-49c0-b85b-417d76618ac3.xml', 
#                               cim_profile=cim_profile, iec61970_301=8)
# SFO2_connection = RDFlibConnection(params)
# SFO2_network = FeederModel(connection=SFO2_connection, container=SFO2_feeder,
#                                 distributed=False)


For this demonstration, the Blazegraph database used by GridAPPS-D will be used for roundtrip testing. During these tests, each of the substation models developed for Grid Atlas will be added to the database. Connectivity and total load within the test feeders will be queried through a simple test.

To connect to the database, we will establish two connections. The first will be used for database bulk upload using the CIM-Loader library previously developed for Grid Atlas. The second will be used for automated database queries using CIMantic Graphs.

In [6]:
# # Connection to Blazegraph Database for CIM-Loader
# from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
# params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
#                                cim_profile=cim_profile, iec61970_301=8)
# bg_loader = BlazegraphLoader(params)

In [7]:
# # Connection to Blazegraph Database for CIMantic Graphs
# from cimgraph.databases import BlazegraphConnection as BlazegraphConnection
# params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
#                                cim_profile=cim_profile, iec61970_301=8)
# blazegraph = BlazegraphConnection(params)

### Create New Main-and-Transfer Substation

To create a new Main and Transfer substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [8]:
from cimbuilder.substation_builder import MainAndTransferSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [9]:
main_transfer = MainAndTransferSubstation(connection=connection,
                                       name="combined_sub", 
                                       base_voltage = 69000)
substation = main_transfer.substation

In [10]:
# xfmr_id = cim.Feeder(mRID = 'B4AF9D95-FD3D-483A-A556-3497A2C912D6')
# params = ConnectionParameters(filename='../test_models/sample_xfmr_69_12kv.xml', 
#                               cim_profile=cim_profile, iec61970_301=8)
# xfmr_connection = RDFlibConnection(params)
# xfmr = FeederModel(connection=xfmr_connection, container=xfmr_id,
#                                 distributed=False)

In [11]:
# xfmr.get_all_edges(cim.PowerTransformerEnd)
# xfmr.pprint(cim.PowerTransformerEnd)

### Create New Sectionalized Bus

To create a new Sectionalized-Bus substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [12]:
from cimbuilder.substation_builder import SectionalizedBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of bus sections is specified by `total_sections` with a default value of two bus sections.

In [13]:
sectionalized = SectionalizedBusSubstation(connection=connection,
                                        name="sectionalized_bus", 
                                        base_voltage = 69000,
                                        total_sections = 2,
                                        substation = substation)

In [14]:
import cimbuilder.object_builder as object_builder

In [15]:
high_side_1 = main_transfer.new_branch(series_number=1)
low_side_1 = sectionalized.new_branch(section_number=1, series_number=1)

transformer = object_builder.new_power_transformer(network=main_transfer.network,
)

To add a feeder to the new substation, the `.new_feeder()` method is called. The `section_number` is used to identify the bus section to which the feeder should be connected. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [16]:
SubBuilder.new_feeder(section_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(section_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

NameError: name 'SubBuilder' is not defined

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [ ]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [ ]:
utils.write_xml(SubBuilder.network, '../test_output/sectionalized.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [ ]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

In [ ]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/sectionalized.xml')

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [ ]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [ ]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

In [ ]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

In [ ]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [ ]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/sectionalized_and_feeders.xml')